# 패키지 임포트

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import nltk
import re
import string
import os
import wordninja
import json
import torch
from itertools import groupby
from operator import itemgetter
from data_preprocessing import *
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# 데이터 로드

In [2]:
train_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-train-stemmed.csv')
valid_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-dev-stemmed.csv')
test_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-test-stemmed.csv')
train_df.head()

,text,edge,intent
0,champion product approv stock split champion p...,champion product approv stock split champion p...,earn
1,comput termin system cpml complet sale comput ...,comput termin system cpml complet sale comput ...,acq
2,cobanco inc cbco year net shr ct dlr net asset...,cobanco inc cbco year net shr ct dlr net asset...,earn
3,intern inc qtr jan oper shr loss two ct profit...,intern inc qtr jan oper shr loss two ct profit...,earn
4,brown forman inc bfd qtr net shr dlr ct net ml...,brown forman inc bfd qtr net shr dlr ct net ml...,earn


In [3]:
# train_df와 valid_df 결합
train_val_df = pd.concat([train_df, valid_df])

# 전처리

In [4]:
train_val_df['text']=train_val_df['text'].apply(clean_text)
test_df['text']=test_df['text'].apply(clean_text)

In [5]:
x_train=list(train_val_df['text'])
y_train=list(train_val_df['intent'])
x_test=list(test_df['text'])
y_test=list(test_df['intent'])

In [6]:
to_txt=x_train+x_test

In [7]:
to_txt=limit_words(to_txt,300)

# 사전 학습 BERT 불러오기

In [8]:
def split_continuous_integers(lst):                                      #연속된 정수리스트를 split 해주는 함수
    for k, g in groupby(enumerate(lst), lambda i_x:i_x[0]-i_x[1]):
        yield list(map(itemgetter(1), g))
        
def add_previous_number(lst):                                            #최초 서브토큰 인덱스 추가
    return [[sub_lst[0] - 1] + sub_lst for sub_lst in lst]

def bert_word_embedding(text):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text) #서브토큰화
    indexed_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    # Python list를 PyTorch tensor로 변환하기 
    tokens_tensor = torch.tensor([indexed_text])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors )
    token_embeddings = torch.stack(encoded_layers, dim=0) #12개 레이어 쌓기 # torch.Size([12, 1, 22, 768])
    token_embeddings = torch.squeeze(token_embeddings, dim=1) #배치 차원 제거 # torch.Size([12, 22, 768])
    token_embeddings = token_embeddings.permute(1,0,2) #위치 변환 # torch.Size([22, 12, 768])
    
    token_vecs_sum = []
    for token in token_embeddings:                #인코더의 마지막 4개의 은닉 상태를 합쳐 최종 벡터(4개 합친 것이 성능이 가장 좋음)
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(np.array(sum_vec))
    
    token_vecs_sum=np.array(token_vecs_sum)
    #서브토큰을 결합해 단어에 대한 임베딩 벡터를 얻자
    subword_indices = [i for i, token in enumerate(tokenized_text) if '##' in token]
    index_list = add_previous_number(list(split_continuous_integers(subword_indices))) #index_list는 서브토큰에 해당하는 인덱스를 한 리스트에 묶어줌
    
    new_token_vecs_sum = []
    last_index = 0

    for subword_inx_list in index_list:
        # 이전 인덱스와 현재 인덱스 그룹 사이의 벡터를 추가합니다.
        new_token_vecs_sum.extend(token_vecs_sum[last_index:subword_inx_list[0]])

        # 현재 인덱스 그룹에 해당하는 벡터의 평균을 계산하고 추가합니다.
        avg_vecs = np.mean(token_vecs_sum[subword_inx_list], axis=0)
        new_token_vecs_sum.append(avg_vecs)

        # 마지막 인덱스를 업데이트합니다.
        last_index = subword_inx_list[-1] + 1

    # 마지막 인덱스 이후의 벡터를 추가합니다.
    new_token_vecs_sum.extend(token_vecs_sum[last_index:])

    # 결과를 numpy array로 변환합니다.
    new_token_vecs_sum = np.array(new_token_vecs_sum[1:-1])
    
    return new_token_vecs_sum

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 미리 학습된 모델(가중치) 불러오기
model = BertModel.from_pretrained('bert-base-uncased')

text = to_txt[5]

marked_text = "[CLS] " + text + " [SEP]"

tokenized_text = tokenizer.tokenize(marked_text) #서브토큰화

indexed_text = tokenizer.convert_tokens_to_ids(tokenized_text)

segments_ids = [1] * len(tokenized_text)

# Python list를 PyTorch tensor로 변환하기 
tokens_tensor = torch.tensor([indexed_text])
segments_tensors = torch.tensor([segments_ids])

with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors )
    
token_embeddings = torch.stack(encoded_layers, dim=0) #12개 레이어 쌓기 # torch.Size([12, 1, 22, 768])
token_embeddings = torch.squeeze(token_embeddings, dim=1) #배치 차원 제거 # torch.Size([12, 22, 768])
token_embeddings = token_embeddings.permute(1,0,2) #위치 변환 # torch.Size([22, 12, 768])

token_vecs_sum = []

for token in token_embeddings:
    
    sum_vec = torch.sum(token[-4:], dim=0)
    
    token_vecs_sum.append(np.array(sum_vec))
    

token_vecs_sum=np.array(token_vecs_sum)

token_vecs_sum=bert_word_embedding(text)
token_vecs_sum.shape

(140, 768)

# 문서 단위 임베딩

In [10]:
x_train=to_txt[:5484]
x_test=to_txt[5484:]

In [11]:
except_lst=[]
for i in range(len(x_train)):
    try:
        text = to_txt[i]
        embed_matrix=bert_word_embedding(text)       
        #임베딩된 문서 임베딩 행렬을 저장
        np.save('./bert_embedding/train(bert)/doc%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        except_lst.append(i)   #sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(37, 768)
(128, 768)
(33, 768)
(70, 768)
(26, 768)
(140, 768)
(131, 768)
(40, 768)
(39, 768)
(19, 768)
(42, 768)
(26, 768)
(52, 768)
(54, 768)
(228, 768)
(110, 768)
(48, 768)
(27, 768)
(15, 768)
(101, 768)
(128, 768)
(39, 768)
(24, 768)
(45, 768)
(47, 768)
(17, 768)
(64, 768)
(50, 768)
(31, 768)
(16, 768)
(15, 768)
(29, 768)
(15, 768)
(26, 768)
(43, 768)
(29, 768)
(17, 768)
(300, 768)
(48, 768)
(72, 768)
(38, 768)
(63, 768)
(33, 768)
(66, 768)
(31, 768)
(7, 768)
(88, 768)
(29, 768)
(63, 768)
(140, 768)
(36, 768)
(14, 768)
(39, 768)
(16, 768)
(17, 768)
(259, 768)
(109, 768)
(29, 768)
(43, 768)
(28, 768)
(16, 768)
(65, 768)
(97, 768)
(48, 768)
(26, 768)
(15, 768)
(33, 768)
(20, 768)
(15, 768)
(34, 768)
(29, 768)
(37, 768)
(22, 768)
(28, 768)
(21, 768)
(25, 768)
(12, 768)
(46, 768)
(50, 768)
(14, 768)
(54, 768)
(21, 768)
(44, 768)
(66, 768)
(58, 768)
(44, 768)
(76, 768)
(172, 768)
(78, 768)
(271, 768)
(254, 768)
(89, 768)
(206, 768)
(300, 768)
(206, 768)
(122, 768)
(84, 768)
(61, 768)
(24

In [12]:
#제외된 문서가 존재할 경우 아래 셀에 있는 코드의 주석을 풀고 입력 시퀀스의 길이를 줄인 후 재시도

print('제외된 문서의 수: ',len(except_lst))

with open('except_lst.pkl', 'wb') as file:
    pickle.dump(except_lst, file)

print("except_lst를 피클 파일로 저장했습니다.")

제외된 문서의 수:  0
except_lst를 피클 파일로 저장했습니다.


In [13]:
except_test_lst=[]

for i in range(len(x_test)):
    try:
        text = to_txt[i+5484]
        embed_matrix=bert_word_embedding(text)
        #임베딩된 문서 임베딩 행렬을 저장
        np.save('./bert_embedding/test(bert)/test%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        except_test_lst.append(i)   #sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(300, 768)
(68, 768)
(114, 768)
(300, 768)
(60, 768)
(67, 768)
(70, 768)
(102, 768)
(131, 768)
(63, 768)
(47, 768)
(117, 768)
(167, 768)
(9, 768)
(100, 768)
(49, 768)
(127, 768)
(58, 768)
(111, 768)
(146, 768)
(117, 768)
(300, 768)
(62, 768)
(54, 768)
(51, 768)
(78, 768)
(15, 768)
(95, 768)
(6, 768)
(79, 768)
(19, 768)
(77, 768)
(6, 768)
(135, 768)
(27, 768)
(15, 768)
(31, 768)
(7, 768)
(40, 768)
(13, 768)
(30, 768)
(23, 768)
(20, 768)
(109, 768)
(51, 768)
(18, 768)
(254, 768)
(40, 768)
(126, 768)
(13, 768)
(195, 768)
(113, 768)
(15, 768)
(36, 768)
(36, 768)
(21, 768)
(14, 768)
(34, 768)
(55, 768)
(14, 768)
(6, 768)
(12, 768)
(7, 768)
(104, 768)
(13, 768)
(39, 768)
(50, 768)
(11, 768)
(46, 768)
(37, 768)
(86, 768)
(24, 768)
(23, 768)
(15, 768)
(123, 768)
(64, 768)
(40, 768)
(17, 768)
(98, 768)
(49, 768)
(55, 768)
(35, 768)
(15, 768)
(98, 768)
(24, 768)
(8, 768)
(5, 768)
(7, 768)
(14, 768)
(16, 768)
(18, 768)
(13, 768)
(37, 768)
(164, 768)
(6, 768)
(60, 768)
(70, 768)
(34, 768)
(259, 76

(189, 768)
(185, 768)
(300, 768)
(213, 768)
(151, 768)
(115, 768)
(167, 768)
(213, 768)
(55, 768)
(300, 768)
(300, 768)
(115, 768)
(300, 768)
(290, 768)
(103, 768)
(110, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(287, 768)
(126, 768)
(121, 768)
(51, 768)
(300, 768)
(300, 768)
(267, 768)
(128, 768)
(119, 768)
(54, 768)
(40, 768)
(126, 768)
(300, 768)
(200, 768)
(274, 768)
(300, 768)
(125, 768)
(300, 768)
(300, 768)
(300, 768)
(62, 768)
(300, 768)
(151, 768)
(279, 768)
(300, 768)
(280, 768)
(146, 768)
(210, 768)
(67, 768)
(226, 768)
(191, 768)
(134, 768)
(145, 768)
(300, 768)
(300, 768)
(132, 768)
(291, 768)
(76, 768)
(149, 768)
(228, 768)
(52, 768)
(235, 768)
(92, 768)
(60, 768)
(163, 768)
(121, 768)
(300, 768)
(231, 768)
(51, 768)
(300, 768)
(300, 768)
(108, 768)
(123, 768)
(116, 768)
(236, 768)
(143, 768)
(114, 768)
(300, 768)
(125, 768)
(65, 768)
(139, 768)
(151, 768)
(136, 768)
(292, 768)
(209, 768)
(113, 768)
(188, 768)
(142, 768)
(147, 768)
(63, 768)
(263, 768)
(300, 768)
(

(120, 768)
(144, 768)
(223, 768)
(129, 768)
(106, 768)
(83, 768)
(145, 768)
(130, 768)
(300, 768)
(180, 768)
(138, 768)
(123, 768)
(181, 768)
(133, 768)
(118, 768)
(236, 768)
(248, 768)
(139, 768)
(92, 768)
(157, 768)
(166, 768)
(78, 768)
(185, 768)
(110, 768)
(189, 768)
(300, 768)
(300, 768)
(300, 768)
(207, 768)
(300, 768)
(300, 768)
(98, 768)
(128, 768)
(118, 768)
(178, 768)
(300, 768)
(300, 768)
(134, 768)
(84, 768)
(29, 768)
(288, 768)
(211, 768)
(266, 768)
(137, 768)
(110, 768)
(164, 768)
(300, 768)
(66, 768)
(96, 768)
(144, 768)
(300, 768)
(287, 768)
(300, 768)
(90, 768)
(300, 768)
(112, 768)
(51, 768)
(100, 768)
(128, 768)
(300, 768)
(300, 768)
(218, 768)
(50, 768)
(300, 768)
(152, 768)
(164, 768)
(180, 768)
(179, 768)
(272, 768)
(133, 768)
(300, 768)
(126, 768)
(121, 768)
(300, 768)
(300, 768)
(284, 768)
(150, 768)
(154, 768)
(53, 768)
(166, 768)
(227, 768)
(270, 768)
(57, 768)
(130, 768)
(45, 768)
(300, 768)
(172, 768)
(62, 768)
(168, 768)
(300, 768)
(300, 768)
(149, 768)
(82

(224, 768)
(48, 768)
(178, 768)
(300, 768)
(174, 768)
(95, 768)
(129, 768)
(300, 768)
(131, 768)
(260, 768)
(180, 768)
(300, 768)
(55, 768)
(300, 768)
(94, 768)
(120, 768)
(226, 768)
(125, 768)
(300, 768)
(300, 768)
(149, 768)
(148, 768)
(199, 768)
(297, 768)
(161, 768)
(300, 768)
(153, 768)
(174, 768)
(219, 768)
(300, 768)
(121, 768)
(174, 768)
(195, 768)
(300, 768)
(300, 768)
(179, 768)
(176, 768)
(144, 768)
(90, 768)
(300, 768)
(120, 768)
(275, 768)
(300, 768)
(210, 768)
(124, 768)
(247, 768)
(300, 768)
(194, 768)
(183, 768)
(299, 768)
(49, 768)
(95, 768)
(60, 768)
(133, 768)
(37, 768)
(300, 768)
(116, 768)
(300, 768)
(182, 768)
(300, 768)
(153, 768)
(108, 768)
(54, 768)
(87, 768)
(300, 768)
(42, 768)
(233, 768)
(161, 768)
(300, 768)
(169, 768)
(79, 768)
(300, 768)
(121, 768)
(76, 768)
(112, 768)
(300, 768)
(67, 768)
(137, 768)
(44, 768)
(299, 768)
(300, 768)
(300, 768)
(249, 768)
(110, 768)
(219, 768)
(131, 768)
(284, 768)
(300, 768)
(177, 768)
(79, 768)
(267, 768)
(121, 768)
(72, 

In [14]:
#제외된 문서가 존재할 경우 아래 셀에 있는 코드의 주석을 풀고 입력 시퀀스의 길이를 줄인 후 재시도

print('제외된 문서의 수: ',len(except_test_lst))

with open('except_test_lst.pkl', 'wb') as file:
    pickle.dump(except_test_lst, file)

print("except_test_lst를 피클 파일로 저장했습니다.")

제외된 문서의 수:  0
except_test_lst를 피클 파일로 저장했습니다.
